In [1]:
import PIL
import gc
import torch
import torchvision
import os

import numpy as np
import matplotlib.pyplot as plt  
import torch.nn as nn 
import torch.optim as optim
import torch.nn.functional as F

from torchvision import datasets, transforms
from torch.utils.data import Subset
from IPython.core.display import display, HTML
from numpy.random import RandomState
from wide_resnet import WideResNet
from auto_augment import AutoAugment, Cutout
from efficientnet_pytorch import EfficientNet
from cifar_loader import SmallSampleController


# display(HTML("<style>.container { width:40% !important; }</style>"))


In [2]:
def train(model, device, train_loader, optimizer, epoch, display=True):
    """
    Summary: Implements the training procedure for a given model
    == params ==
    model: the model to test
    device: cuda or cpu 
    optimizer: the optimizer for our training
    train_loader: dataloader for our train data
    display: output flag
    == output ==
    the mean train loss, the train accuracy
    """
    
    lossTracker = []
    
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        lossTracker.append(loss.detach())
        
    lossTracker = [x.item()]
    meanLoss = np.mean(lossTracker)
    accuracy = 100. * batch_idx / len(train_loader)
    if display:
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
          epoch, batch_idx * len(data), len(train_loader.dataset),
          accuracy, meanLoss))
    return accuracy, meanLoss



def test(model, device, test_loader,verbose=True):
    """
    Summary: Implements the testing procedure for a given model
    == params ==
    model: the model to test
    device: cuda or cpu 
    test_loader: dataloader for our test data
    verbose: output flag
    == output ==
    the mean test loss, the test accuracy
    """
    
    lossTracker = []
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.cross_entropy(output, target, size_average=False).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
            

    mean_test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    
    if verbose: print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        mean_test_loss, correct, len(test_loader.dataset),
        accuracy))
        
    return mean_test_loss, accuracy


def checkTest(model,device,valSets,valTracker,verbose=True):
    """
    Summary: checks the test accuracy, prints, and saves statistics
    """
    tempAcc = []
    tempLoss = []
    for val_loader in valSets:
        acc,loss = test(model, device, val_loader,verbose = False)
        tempAcc.append(acc)
        tempLoss.append(loss)
        
    accMean = np.mean(tempAcc)
    accStd = np.std(tempAcc)
    
    lossMean = np.mean(tempLoss)
    if verbose:
        print('[Trained for {} epochs and tested on {} sets of 2000 images]\
        Avg Acc: {:.2f} +- {:.2f}'.format(epochNum,VALIDATION_SET_NUM,accMean,accStd))
        
    valTracker["allLoss"].extend(tempLoss)
    valTracker["allAcc"].extend(tempAcc)
    valTracker["meanLoss"].append(meanLoss)
    valTracker["meanAcc"].append(meanAcc)
    valTracker["stdAcc"].append(stdAcc)





In [3]:
def getLatexRow(testSize,arch,acc,epoch,lr,dataAug="Nothing",title=False):
    """
    Summary: generates one row of latex for a results table
    """
    categories = ["Architecture","Accuracy","Epochs","learning rate","testSize","data augmentation"]
    row = [str(arch),str(round(acc,3)),str(epoch),str(lr),str(testSize),str(dataAug)]
    
    if title:
        c = "&".join(categories)
        r = "&".join(row)
        return "{}\\\\\n{}\\\\\n".format(c,r)
    else:
        r = "&".join(row)
        return "{}\\\\\n".format(r)
    
    
def plot(list_,xlab,ylab,title,color,label,save=False):
    """
    Summary: plots the given list of numbers against its idices and 
    allows for high resolution saving
    """
    
    fig = plt.figure()
    plt.title(title)
    plt.xlabel(xlab)
    plt.ylabel(ylab)
    plt.plot(
        list_,[(x+1) for x in range(len(list))],
        color=color,marker=".",
        label=label)
    plt.legend()
    
    if save:
        plt.savefig("{}.pdf".format(title))
        os.system("pdftoppm -png -r 300 {}.pdf {}".format(title))
    plt.show()
    
    

In [4]:
def getModel(model_name):
    if "wide" in model_name.lower():
        return WideResNet(28, 10, num_classes=10)
    elif "fix" in model_name.lower():
        return EfficientNet.from_pretrained(model_name) # change to not be pretrained
    
    
def getOptimizer(optimizer_name,model):
    if "adam" in  optimizer_name.lower():
        return torch.optim.SGD(model.parameters(), 
                                  lr=0.09, momentum=0.9,
                                  weight_decay=0.0005)
    elif "sdg" in optimizer_name.lower():
        return torch.optim.Adam(model.parameters(), 
                              lr=0.001, weight_decay=0)
        
    

In [5]:
torch.cuda.empty_cache()
gc.collect()


EPOCH_NUM = 250
TRAIN_SAMPLE_NUM = 100
VAL_SAMPLE_NUM = 2000
BATCH_SIZE = 256
VALIDATION_SET_NUM = 10
AUGMENT = True




normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                  std=[0.229, 0.224, 0.225])
if AUGMENT:
    dataAugmentation = [ 
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        AutoAugment(),
        Cutout()
    ]
    augment = "Crop,Flip,AutoAugment,Cutout"
else: 
    dataAugmentation = []
    augment = "Nothing"



transform_train = transforms.Compose(dataAugmentation + [transforms.ToTensor(), normalize]) 
transform_val = transforms.Compose([transforms.ToTensor(), normalize]) #careful to keep this one same

cifar_train = datasets.CIFAR10(root='.',train=True, transform=transform_train, download=True)
cifar_val = datasets.CIFAR10(root='.',train=True, transform=transform_val, download=True)

ss = SmallSampleController(numClasses=10,trainSampleNum=TRAIN_SAMPLE_NUM, # abstract the data-loading procedure
                           valSampleNum=VAL_SAMPLE_NUM, batchSize=BATCH_SIZE, 
                           multiplier=VALIDATION_SET_NUM, trainDataset=cifar_train, 
                           valDataset=cifar_val)
    
    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_data, valSets = ss.generateNewSet(device,valMultiplier = VALIDATION_SET_NUM) #Sample from datasets


model = getModel("WideResNet").cuda()
optimizer = getOptimizer("adam",model)

print(' => Total params: %.2fM' % (sum(p.numel() for p in model.parameters()) / 1000000.0))        

trainTracker = {"meanLoss":[],"accuracy":[]}
valTracker = {"allLoss":[],"allAcc":[],"meanLoss":[],"meanAcc":[],"stdAcc":[]}


print("Begin Train for {} epochs".format(EPOCH_NUM))
for epoch in range(EPOCH_NUM):
    train(model, device, train_data[0], optimizer, epoch, display=True)
    if (epoch+1) % 25 == 0:
        checkTest(model,device,valSets,valTracker,verbose=True)
        
            
        
        

Files already downloaded and verified
Files already downloaded and verified
[5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000, 5000]


/home/benjamin/venv/torch11/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.

In [ ]:
plot(list_=trainTracker["meanLoss"],xlab="Mean Train Loss",
    ylab="Epochs",title="Mean Train Loss over Epochs",
    color="#243A92",label="mean train loss",save=True)

plot(list_=trainTracker["accuracy"],xlab="Train Accuracy",
    ylab="Epochs",title="Train Accuracy Over Epochs",
    color="#34267E",label="Train Accuracy",save=True)

In [ ]:
plot(list_=valTracker["meanLoss"],xlab="Mean Val Loss",
    ylab="25 Epochs",title="Mean Val Loss over 25 Epochs",
    color="#243A92",label="mean val loss",save=True)

plot(list_=valTracker["meanAcc"],xlab="Val Accuracy",
    ylab="25 Epochs",title="Val Accuracy Over 25 Epochs",
    color="#34267E",label="Val Accuracy",save=True)

plot(list_=valTracker["stdAcc"],xlab="Val Accuracy Standard Deviation",
    ylab="25 Epochs",title="Val Accuracy Standard Deviation Over 25 Epochs",
    color="#34267E",label="Val Accuracy SD",save=True)

plot(list_=valTracker["allLoss"],xlab="Val Loss",
    ylab="25 Epochs",title="Val Loss Over 25 Epochs",
    color="#34267E",label="Val Loss",save=True)

plot(list_=valTracker["allAcc"],xlab="Val Accuracy",
    ylab="25 Epochs",title="Val Accuracy Over 25 Epochs",
    color="#34267E",label="Val Accuracy",save=True)